In [1]:
import os

# TODO:根据自己的需求，屏蔽或启用GPU
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

import torch 
from torch import nn
import torchvision
import torch.utils.data as Data
import math

import numpy as np

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# TODO:把包的绝对路径改成你自己放包的路径
os.sys.path.append(r'F:\work\jupyter\Transfer Learning')
from tools import mmd
from tools import torch_plot
from tools import data_loader as loader
from tools import resnet 

# 全局配置

包括文件路径，迭代次数，batch_size等

In [2]:
# 全局配置，包括文件路径等
base_dir = r"F:\work\jupyter\Transfer Learning\Dateset"
batch_size = 64
epochs = 50
source = '0HP'
target = '2HP'

In [3]:
target

'2HP'

# 加载数据

In [3]:
source_data, source_label = loader.load_data(base_dir, source, batch_size=batch_size)
target_data, target_label = loader.load_data(base_dir, target, batch_size=batch_size)

# 构建网络

In [10]:
class Classificator(nn.Module):
    # 全连接层构成的分类器
    def __init__(self, num_classes=1):
        super().__init__()
        # resnet18的输出是512维的
        self.fc = nn.Sequential(
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 32 ),
            nn.ReLU(),
            nn.Linear(32 , num_classes)
        )
        
    def forward(self, inputs):
        return self.fc(inputs)


class IWAN(nn.Module):
    def __init__(self):
        super().__init__()
        self.source_feature_extractor = resnet.resnet18()
        self.target_feature_extractor = resnet.resnet18()
        self.discriminator = Classificator()
        self.classificator = Classificator(10)
        self.weight = None
        
    def forward(self, source, target=None):
        if self.training:
            # 特征提取：
            source_feature = self.source_feature_extractor(source)
            target_feature = self.target_feature_extractor(target)
            
            # 经过领域鉴别器，注意这里是一个二分类，激活函数要使用sigmoid,
            # 输出的结果可以看作是概率
            ps = torch.sigmoid(self.discriminator(source_feature))
            pz = torch.sigmoid(self.discriminator(target_feature))
            
            # 将源域数据加权，并送入分类器
            self.weight = 1/(ps/pz + 1)
            weighted_source_feature = source_feature * self.weight
            
            # 源域数据送入分类器
            source_classificated = self.classificator(weighted_source_feature)
            
            return source_classificated, ps, pz
        
        # 测试时走这条路    
        else:
            # 特征提取
            target_feature = self.target_feature_extractor(target)
            # 加权
            weighted_target_feature = target_feature * self.weight
            # 分类
            target_classificated = self.classificator(weighted_target_feature)
            
            return target_classificated
        

In [16]:
def train(dataloader, model, optimizer):
    """
    dataloader:(source_data, source_label, target_data)构成的元组
    """
    size_source = len(dataloader[0].dataset)    # 源域的样本总量
    num_batches = len(dataloader[0])            # 源域的批数
    
    # 定义损失函数
    loss_fn_classification = nn.CrossEntropyLoss()     # 分类损失函数
    loss_fn_dicriminate = nn.BCELoss(reduction='mean') # 领域鉴别损失
    
    # 训练
    model.train()
    train_loss, correct, discriminate_loss = 0, 0, 0
    batch = 0
    for sourceData, sourceLabel, targetData in zip(dataloader[0], dataloader[1], dataloader[2]):
        X_source, y = sourceData.to(device), sourceLabel.to(device)
        X_target = targetData.to(device)
        
        # 前向传播以及损失计算
        pred, ps, pz = model(X_source, X_target)                   # 分类器输出, Ps, Pz
        classification_loss = loss_fn_classification(pred, y)      # 分类损失
        print(ps)
        discriminate_loss_source = loss_fn_dicriminate(ps, torch.ones((batch_size, 1)))  # 源域领域鉴别损失
        discriminate_loss_tatget = loss_fn_dicriminate(pz, torch.zeros((batch_size, 1))) # 目标域领域鉴别损失
        
        train_loss += classification_loss.item() + \
                        discriminate_loss_source.item() + \
                        discriminate_loss_tatget.item()
        
        correct += (pred.argmax(1) == y).type(torch.float).sum().item()
        
        discriminate_loss += discriminate_loss_source.item() + discriminate_loss_tatget.item()
        
        loss = classification_loss + discriminate_loss_source + discriminate_loss_tatget
        # 反向传播
        optimizer.zero_grad()
        loss.backward()
#         discriminate_loss_source.backward()
#         discriminate_loss_tatget.backward()
        optimizer.step()
        batch += 1
        
        if batch % 10 == 0:
            loss, current = loss.item(), batch * len(X_source)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size_source:>5d}]")
        
    train_loss /= num_batches
    correct /= size_source 
    discriminate_loss /= num_batches
    
    return train_loss, correct, discriminate_loss


# def test(dataloader, model, loss_fn):
#     """
#     dataloader:(target_data, target_label)构成的元组
#     """
#     size = len(dataloader[0].dataset)
#     num_batches = len(dataloader[0])
#     model.eval()
#     test_loss, correct, mmd_loss_ret = 0, 0, 0
#     with torch.no_grad():
#         for X, y in zip(dataloader[0], dataloader[1]):
#             X, y = X.to(device), y.to(device)
#             pred, mmd_loss = model(X)
#             classification_loss = loss_fn(pred, y)      # 分类损失
#             loss = mmd_loss + classification_loss       # 总的损失
            
#             test_loss += classification_loss.item()
#             mmd_loss_ret += mmd_loss
    
#             correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            
#     test_loss /= num_batches
#     correct /= size
#     mmd_loss_ret /= num_batches
    
#     print(f"Test Error: \n Accuracy: {(100 * correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
#     return test_loss, correct


In [17]:
model = IWAN().to(device)

optimizer = torch.optim.RMSprop(model.parameters(), lr=1e-3, momentum=0.9)
# print(model)   

epochs = 100
train_losses = []
train_accs = []
train_mmd_losses = []

test_losses = []
test_accs = []

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loss, train_acc, train_mmd_loss = train((source_data, source_label, target_data), model, optimizer)
    train_losses.append(train_loss)
    train_accs.append(train_acc)
    train_mmd_losses.append(train_mmd_loss)
    
    # 测试模型
#     test_loss, test_acc = test((target_data, target_label), model, loss_fn)
#     test_losses.append(test_loss)
#     test_accs.append(test_acc)

print("Done!")    

Epoch 1
-------------------------------
tensor([[0.4999],
        [0.5033],
        [0.5010],
        [0.5019],
        [0.5031],
        [0.5005],
        [0.5042],
        [0.5038],
        [0.5032],
        [0.5007],
        [0.4986],
        [0.4959],
        [0.4986],
        [0.5032],
        [0.5021],
        [0.5053],
        [0.5099],
        [0.4996],
        [0.5071],
        [0.5060],
        [0.4990],
        [0.5025],
        [0.4939],
        [0.5059],
        [0.4986],
        [0.5009],
        [0.5047],
        [0.4996],
        [0.5030],
        [0.5036],
        [0.5025],
        [0.5097],
        [0.5049],
        [0.5020],
        [0.5050],
        [0.5037],
        [0.5037],
        [0.5021],
        [0.5051],
        [0.4937],
        [0.5015],
        [0.5034],
        [0.5020],
        [0.5044],
        [0.5042],
        [0.4943],
        [0.5060],
        [0.5098],
        [0.5045],
        [0.5014],
        [0.4990],
        [0.5031],
        [0.5074],
      

tensor([[0.9999],
        [1.0000],
        [0.9998],
        [0.9999],
        [0.9999],
        [0.9999],
        [0.9999],
        [0.9996],
        [0.9999],
        [0.9999],
        [0.9999],
        [0.9993],
        [0.9999],
        [0.9999],
        [0.9999],
        [0.9999],
        [0.9999],
        [0.9999],
        [0.9999],
        [0.9998],
        [0.9999],
        [0.9996],
        [0.9999],
        [0.9999],
        [0.9999],
        [0.9999],
        [0.9892],
        [0.9999],
        [0.9999],
        [0.9998],
        [0.9999],
        [0.9999],
        [0.9999],
        [0.9999],
        [0.9998],
        [0.9999],
        [0.9999],
        [0.9998],
        [0.9999],
        [0.9999],
        [0.9999],
        [0.9999],
        [0.9999],
        [0.9998],
        [0.9999],
        [1.0000],
        [1.0000],
        [0.9999],
        [0.9999],
        [0.9999],
        [0.9999],
        [0.9999],
        [0.9999],
        [0.9999],
        [0.9999],
        [0

RuntimeError: all elements of input should be between 0 and 1